In [ ]:
import csv

_FIELD_NAMES = [
    'song_title',
    'song_hash',
    'song_difficulty',
    'song_meter',
    'members_name',
    'score',
    'lamp'
]

song_hash_chars = {}

src = '3ic_data/3ic_data_20220805.csv'
with open(src, 'r', encoding='utf-8') as fp:
    reader = csv.DictReader(fp, fieldnames=_FIELD_NAMES)
    for row in reader:
        for c in row['song_hash']:
            if c in song_hash_chars:
                song_hash_chars[c] += 1
            else:
                song_hash_chars[c] = 0

print(len(song_hash_chars))

In [ ]:
for c in sorted(song_hash_chars):
    print(f'{c:3s} {ord(c):3d} 0x{ord(c):02X}')

In [ ]:
def a8e(s: str) -> str:
    s = s.strip()
    s2 = [c for c in s.lower() if (c >= 'a' and c <= 'z') or (c >= '0' and c <= '9')]
    l = len(s2)
    return f'{s[0]}{l-2}{s[-1]}'

for t in ['Puberty Dysthymia', 'Cytokinesis', 'Dance Dance Revolution']:
    print(a8e(t))


In [ ]:
v = [chr(v + ord('A')) for v in range(5)]

load_order = []

for i_row, a in enumerate(v):
    tri = v[i_row:]
    if i_row % 2 == 1:
         tri.reverse()
    for i_col, b in enumerate(tri):
        load_order.append( (a, b) )

load_replacement = [load_order[0]]
for prev, next in zip(load_order[:-1], load_order[1:]):
    repl = [a != b and b or None for a, b in zip(prev, next)]
    load_replacement.append(tuple(repl))

print(load_replacement)


In [ ]:
_PARTITION_SIZE = 3
_TOTAL_LENGTH = 26
_EXCLUDE_DIAGONAL = True
_EXPLICIT_PAIRS = False

v = [chr(v + ord('A')) for v in range(_TOTAL_LENGTH)]
v_part = [v[i:i+_PARTITION_SIZE] for i in range(0, _TOTAL_LENGTH, _PARTITION_SIZE)]
print(v_part)

load_order = []

load_replacement = []
exc_diag = _EXCLUDE_DIAGONAL and 1 or 0
for i_row in range(0, len(v_part), 2):
    # (i, i) --> (i, n)
    for i_col in range(i_row + exc_diag, len(v_part)):
        if _EXPLICIT_PAIRS or (i_col == i_row + exc_diag):
            load_replacement.append( (i_row, i_col) )
        else:
            load_replacement.append( (None, i_col) )

    # (i+1, i+1) <-- (i+1, n)
    if i_row + 1 >= len(v_part):
        break
    for i_col in range(len(v_part) - 1, i_row + exc_diag, -1):
        if i_col == len(v_part) - 1:
            load_replacement.append( (i_row + 1, _EXPLICIT_PAIRS and i_col or None) )
        else:
            load_replacement.append( (_EXPLICIT_PAIRS and (i_row + 1) or None, i_col) )

print(load_replacement)


In [ ]:
import requests
import logging
import os
import json
from datetime import datetime as dt

def timestamp():
    return dt.utcnow().strftime('%Y%m%d-%H%M%S-%f')[:-3]
p = os.path.join('itl2023_data', dt.utcnow().strftime('%Y%m%d'))

# Set up logging
logging.getLogger().handlers.clear()
log_stamp = timestamp()
log_path = os.path.join(p, f'scobility-scrape-{log_stamp}.log')
log_fmt = logging.Formatter(
    '[%(asctime)s.%(msecs)03d] %(levelname)-8s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logging.basicConfig(
    filename=log_path,
    encoding='utf-8',
    level=logging.INFO
)
logging.getLogger().addHandler(logging.StreamHandler())
for handler in logging.getLogger().handlers:
    handler.setFormatter(log_fmt)

In [ ]:
# Chart query
charts = {}
strikes = []
for i in range(10000):
    r = requests.get(f'https://itl2023.groovestats.com/api/chart/{i}')
    j = r.json()
    if not j.get('success', False):
        logging.warning(f"{i:4d}: {j.get('message', '')}")
        strikes.append(i)
        if len(strikes) > 5:
            break
    else:
        strikes = []
        charts[i] = j.get('data', {})
        full_name = f"{charts[i].get('artist')} - \"{charts[i].get('title')}\""
        logging.info(f'{i:4d}: {full_name}')

if not os.path.exists(p):
    os.makedirs(p)
with open(os.path.join(p, 'charts.json'), 'w', encoding='utf-8') as fp:
    json.dump(charts, fp)

In [ ]:
p_entrants = os.path.join(p, 'entrant_info')
if not os.path.exists(p_entrants):
    os.makedirs(p_entrants)

# Entrant query
entrants = {}
strikes = []
for i in range(10000):
    r = requests.get(f'https://itl2023.groovestats.com/api/entrant/{i}')
    j = r.json()
    if not j.get('success', False):
        logging.warning(f"{i:4d}: {j.get('message', '')}")
        strikes.append(i)
        if len(strikes) > 5:
            break
    else:
        strikes = []
        entrants[i] = j.get('data', {})
        full_name = f"{entrants[i]['entrant']['name']} (ITL #{entrants[i]['entrant']['id']}, GS #{entrants[i]['entrant']['membersId']})"
        logging.info(f'{i:4d}: {full_name}')

        with open(os.path.join(p_entrants, f'{i}.json'), 'w', encoding='utf-8') as fp:
            json.dump(entrants[i], fp)

In [ ]:
p_scores = os.path.join(p, 'song_scores')
if not os.path.exists(p_scores):
    os.makedirs(p_scores)

with open(os.path.join(p, 'charts.json'), 'r', encoding='utf-8') as fp:
    charts = json.load(fp)

# Entrant query
scores = {}
strikes = []
total = 0
for c in charts.values():
    total += 1
    if total > 10000:
        break

    i = c.get('id', 0)

    r = requests.post(
        f'https://itl2023.groovestats.com/api/score/chartTopScores',
        data={'chartHash': c['hash']}
    )
    j = r.json()
    if not j.get('success', False):
        logging.warning(f"{i:4d} (hash {c['hash']}): {j.get('message', '')}")
        strikes.append(i)
        if len(strikes) > 5:
            break
    else:
        strikes = []
        full_name = f"{c.get('artist')} - \"{c.get('title')}\""
        scores[i] = j.get('data', {}).get('leaderboard', {})
        for s in scores[i]:
            s['chartId'] = i
        logging.info(f"{i:4d} (hash {c['hash']}): {full_name}, {len(scores[i])} scores")

        with open(os.path.join(p_scores, f'{i}.json'), 'w', encoding='utf-8') as fp:
            json.dump({'scores': scores[i]}, fp)

In [ ]:
p_charts = os.path.join(p, 'song_info')
if not os.path.exists(p_charts):
    os.makedirs(p_charts)

with open(os.path.join(p, 'charts.json'), 'r', encoding='utf-8') as fp:
    charts = json.load(fp)

for c in charts.values():
    i = c.get('id', 0)
    
    full_name = f"{c.get('artist')} - \"{c.get('title')}\""
    logging.info(f"{i:4d} (hash {c['hash']}): {full_name}")

    with open(os.path.join(p_charts, f'{i}.json'), 'w', encoding='utf-8') as fp:
        json.dump({'song': c}, fp)

In [ ]:
import os
import json

p = os.path.join('itl2023_data', '20230405')

p_charts = os.path.join(p, 'song_info')
if not os.path.exists(p_charts):
    os.makedirs(p_charts)

with open(os.path.join(p, 'charts.json'), 'r', encoding='utf-8') as fp:
    charts = json.load(fp)

c_meter = sorted(list(charts.values()), key=lambda v: v.get('meter', 0))
for c in c_meter:
    print(f"{c.get('id', 0):4d}: {c.get('meter', 0):2d} worth a max of {c.get('points', 0):5d} pts.")

In [ ]:
from datetime import datetime as dt

x = dt.strptime('2023-03-27T03:38:53.000Z', '%Y-%m-%dT%H:%M:%S.%fZ')

y = x - dt.utcfromtimestamp(0)
print(y.total_seconds())

In [ ]:
import numpy as np

a = np.array([np.nan, 3, 4, np.nan, 5])
x = ~np.isnan(a)
sum(x)

In [ ]:
import hsluv
import numpy as np

colors = [hsluv.hsluv_to_hex([330, saturation, 50]) for saturation in np.linspace(0, 100, 11)]
colors

In [ ]:
from typing import List
def colorize_dates(dates: List[float], alpha: int = 170) -> List[str]:
    dates_days = [round(d / 86400) for d in dates]
    dates_unique = sorted(list(set(dates_days)))
    colors_ref = [hsluv.hsluv_to_hex([345, 100 * saturation**2, 50 * saturation]) + f'{alpha:02x}' for saturation in np.linspace(0, 1, len(dates_unique))]
    dates_match = [dates_unique.index(v) for v in dates_days]
    colors = [colors_ref[i] for i in dates_match]
    return colors

In [ ]:
colorize_dates([300000, 500000, 400000, 300000, 200000, 500000])

In [ ]:
x = np.linspace(1, 0, 11)


In [ ]:
def point_curve(pc, ex: float) -> float:
    if pc == 'itl2023':
        x = ex * 100
        return np.log(min(x, 50) + 1)/np.log(1.1032889141348) + np.power(61, (max(x, 50)-50)/50) - 1
    elif pc == 'itl2022':
        x = ex * 100
        return np.log(min(x, 75) + 1)/np.log(1.0638215)       + np.power(31, (max(x, 75)-75)/25) - 1
    else:   # Unknown or don't care
        return 0

In [ ]:
def point_curve(pc, v: np.ndarray, d1: bool = False) -> np.ndarray:
    if pc == 'itl2023':
        log_base = 1.1032889141348
        pow_base = 61
        inflect = 50
    elif pc == 'itl2022':
        log_base = 1.0638215
        pow_base = 31
        inflect = 75
    else:
        return np.zeros_like(v)

    v_lo = np.clip(v, a_min=None, a_max=inflect)
    v_hi = np.clip(v, a_min=inflect, a_max=None)

    if d1:
        # Not bothering to account for undifferentiable inflection point
        return \
            (v <= inflect) / (np.log(log_base) * (v_lo + 1)) + \
            (v >  inflect) * np.power(pow_base, (v_hi-inflect)/(100-inflect)) * np.log(pow_base) / (100-inflect)
    else:
        return \
            np.log(v_lo + 1) / np.log(log_base) + \
            np.power(pow_base, (v_hi-inflect)/(100-inflect)) - 1

In [ ]:
def point_curve_inv(pc, p: np.ndarray, iterations: int = 15, approach: float = 0.99) -> np.ndarray:
    v = p
    with np.errstate(divide='ignore', invalid='ignore'):
        for i in range(iterations):
            v_next = v - approach * (point_curve(pc, v) - p) / point_curve(pc, v, d1=True)
            v = v_next
            v[np.isnan(v)] = 0
    return v


In [ ]:
from matplotlib import pyplot as plt

x = np.linspace(0, 100, 52)
y2022 = point_curve('itl2022', x)
y2023 = point_curve('itl2023', x)
plt.plot(x, y2022)
plt.plot(x, y2023)
plt.show()

In [ ]:
x = np.linspace(0, 100, 52)
y2022_d1 = point_curve('itl2022', x, d1=True)
y2023_d1 = point_curve('itl2023', x, d1=True)
plt.plot(x, y2022_d1)
plt.plot(x, y2023_d1)
plt.show()

In [ ]:
x = np.linspace(0, 100, 51)
y2022_inv = point_curve_inv('itl2022', x)
y2023_inv = point_curve_inv('itl2023', x)
y2022_test = point_curve('itl2022', y2022_inv)
y2023_test = point_curve('itl2023', y2023_inv)
plt.plot(x, y2022_test-x)
plt.plot(x, y2023_test-x)
plt.show()

In [ ]:
def point_curve_inv(pc, p: np.ndarray) -> np.ndarray:
    # I'm a dumbass this function is entirely invertible
    if pc == 'itl2023':
        log_base = 1.1032889141348
        pow_base = 61
        inflect = 50
    elif pc == 'itl2022':
        log_base = 1.0638215
        pow_base = 31
        inflect = 75
    else:
        return np.zeros_like(p)

    piecewise_border = np.round(np.log(inflect + 1)/np.log(log_base) - 1, decimals=3)
    
    p_lo = np.clip(p, a_min=None, a_max=piecewise_border)
    p_hi = np.clip(p, a_min=piecewise_border, a_max=None)
    
    v = np.zeros_like(p)
    with np.errstate(divide='ignore', invalid='ignore'):
        v[p <= piecewise_border] = np.power(log_base, p[p <= piecewise_border]) - 1
        v[p >  piecewise_border] = (100-inflect)*np.log(p[p > piecewise_border] - piecewise_border)/np.log(pow_base) + inflect
    return v

In [ ]:
x = np.linspace(0, 100, 1001)
y2022_inv = point_curve_inv('itl2022', x)
y2023_inv = point_curve_inv('itl2023', x)
y2022_test = point_curve('itl2022', y2022_inv)-x
y2023_test = point_curve('itl2023', y2023_inv)-x
plt.plot(x, y2022_inv)
plt.plot(x, y2023_inv)
plt.show()
plt.plot(x, y2022_test)
plt.plot(x, y2023_test)
plt.show()
plt.plot(x[np.abs(y2022_test) < 0.1], y2022_test[np.abs(y2022_test) < 0.1])
plt.plot(x[np.abs(y2023_test) < 0.1], y2023_test[np.abs(y2023_test) < 0.1])
plt.show()